# Large-Scale Stochastic Variational GP Regression (CUDA) (w/ KISS-GP)

## Overview

In this notebook, we'll give an overview of how to use Deep Kernel Learning with SKI and stochastic variational regression to rapidly train using minibatches on the `song` UCI dataset, which has hundreds of thousands of training examples.

Stochastic variational inference has several major advantages over the standard regression setting. Most notably, the ELBO used for optimization decomposes in such a way that stochastic gradient descent techniques can be used. See https://arxiv.org/pdf/1411.2005.pdf and https://arxiv.org/pdf/1611.00336.pdf for more technical details of this.

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

## Loading Data

For this example notebook, we'll be using the `3droad` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a **~136 MB** file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

if not os.path.isfile('3droad.mat'):
    print('Downloading \'3droad\' UCI dataset...')
    urllib.request.urlretrieve('https://www.dropbox.com/s/f6ow1i59oqx05pl/3droad.mat?dl=1', '3droad.mat')
    
data = torch.Tensor(loadmat('3droad.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Creating a DataLoader

The next step is to create a torch `DataLoader` that will handle getting us random minibatches of data. This involves using the standard `TensorDataset` and `DataLoader` modules provided by PyTorch.

In this notebook we'll be using a fairly large batch size of 1024 just to make optimization run faster, but you could of course change this as you so choose.

In [3]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=2028, shuffle=True)

## Defining the DKL Feature Extractor

Next, we define the neural network feature extractor used to define the deep kernel. In this case, we use a fully connected network with the architecture `d -> 1000 -> 500 -> 50 -> 2`, as described in the original DKL paper. All of the code below uses standard PyTorch implementations of neural network layers.

In [4]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):           
    def __init__(self):                                      
        super(LargeFeatureExtractor, self).__init__()        
        self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
        self.add_module('bn1', torch.nn.BatchNorm1d(1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 1000))
        self.add_module('bn2', torch.nn.BatchNorm1d(1000))
        self.add_module('relu2', torch.nn.ReLU())                       
        self.add_module('linear3', torch.nn.Linear(1000, 500))
        self.add_module('bn3', torch.nn.BatchNorm1d(500))
        self.add_module('relu3', torch.nn.ReLU())                  
        self.add_module('linear4', torch.nn.Linear(500, 50))       
        self.add_module('bn4', torch.nn.BatchNorm1d(50))
        self.add_module('relu4', torch.nn.ReLU())                  
        self.add_module('linear5', torch.nn.Linear(50, 2))         
                                                             
feature_extractor = LargeFeatureExtractor().cuda()
# num_features is the number of final features extracted by the neural network, in this case 2.
num_features = 2

## Defining the GP Regression Layer

We now define the GP regression module that, intuitvely, will act as the final "layer" of our neural network. In this case, because we are doing variational inference and *not* exact inference, we will be using an `AbstractVariationalGP`. To use grid interpolation for variational inference, we'll be using a `GridInterpolationVariationalStrategy`.

Because the feature extractor we defined above extracts two features, we'll need to define our grid bounds over two dimensions.

See the CIFAR example for using an `AbstractVariationalGP` with an `AdditiveGridInterpolationVariationalStrategy`, which additionally assumes the kernel decomposes additively, which is a strong modelling assumption but allows us to use many more output features from the neural network.

In [5]:
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution, GridInterpolationVariationalStrategy
class GPRegressionLayer(AbstractVariationalGP):
    def __init__(self, grid_size=32, grid_bounds=[(-1, 1), (-1, 1)]):
        variational_distribution = CholeskyVariationalDistribution(num_inducing_points=grid_size*grid_size)
        variational_strategy = GridInterpolationVariationalStrategy(self,
                                                                    grid_size=grid_size,
                                                                    grid_bounds=grid_bounds,
                                                                    variational_distribution=variational_distribution)
        super(GPRegressionLayer, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(
            log_lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(0.001, 1., sigma=0.1, log_transform=True)
        ))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

## Defining the DKL Model

With the feature extractor and GP regression layer defined, we can now define our full model. To do this, we simply create a module whose `forward()` method passes the data first through the feature extractor, and then through the GP regression layer.

The only other interesting feature of the model below is that we use a helper function, `scale_to_bounds`, to ensure that the features extracted by the neural network fit within the grid bounds used for SKI.

In [6]:
class DKLModel(gpytorch.Module):
    def __init__(self, feature_extractor, num_features, grid_bounds=(-1., 1.)):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GPRegressionLayer()
        self.grid_bounds = grid_bounds
        self.num_features = num_features

    def forward(self, x):
        features = self.feature_extractor(x)
        features = gpytorch.utils.grid.scale_to_bounds(features, self.grid_bounds[0], self.grid_bounds[1])
        res = self.gp_layer(features)
        return res

model = DKLModel(feature_extractor, num_features=num_features).cuda()
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()

## Training the Model

The cell below trains the DKL model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE.

Unlike when using the exact GP marginal log likelihood, performing variational inference allows us to make use of stochastic optimization techniques. For this example, we'll do one epoch of training. Given the small size of the neural network relative to the size of the dataset, this should be sufficient to achieve comparable accuracy to what was observed in the DKL paper.

The optimization loop differs from the one seen in our more simple tutorials in that it involves looping over both a number of training iterations (epochs) *and* minibatches of the data. However, the basic process is the same: for each minibatch, we forward through the model, compute the loss (the `VariationalELBO`), call backwards, and do a step of optimization.

In [7]:
model.train()
likelihood.train()

# We'll do 6 epochs of training in this tutorial
num_epochs = 7

# We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters(), 'weight_decay': 1e-3},
    {'params': model.gp_layer.parameters()},
    {'params': likelihood.parameters()},
], lr=0.1)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 5], gamma=0.1)

# Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=train_y.size(0), combine_terms=False)

for i in range(num_epochs):
    scheduler.step()
    # Within each iteration, we will go over each minibatch of data
    for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):        
        optimizer.zero_grad()
        
        # Because the grid is relatively small, we turn off the Toeplitz matrix multiplication and just perform them directly
        # We find this to be more efficient when the grid is very small.
        with gpytorch.settings.use_toeplitz(False):
            output = model(x_batch)
            log_lik, kl_div, log_prior = mll(output, y_batch)
            loss = -(log_lik - kl_div + log_prior)
            print('Epoch %d [%d/%d] - Loss: %.3f [%.3f, %.3f, %.3f]' % (i + 1, minibatch_i, len(train_loader), loss.item(), log_lik.item(), kl_div.item(), log_prior.item()))

        # The actual optimization step
        loss.backward()
        optimizer.step()

Epoch 1 [0/172] - Loss: 176.594 [-176.593, 0.000, -0.000]
Epoch 1 [1/172] - Loss: 152.574 [-152.070, 0.505, -0.000]
Epoch 1 [2/172] - Loss: 151.459 [-150.812, 0.647, -0.000]
Epoch 1 [3/172] - Loss: 145.079 [-144.706, 0.373, -0.000]
Epoch 1 [4/172] - Loss: 129.085 [-128.564, 0.521, -0.000]
Epoch 1 [5/172] - Loss: 103.243 [-102.748, 0.494, -0.000]
Epoch 1 [6/172] - Loss: 94.733 [-94.339, 0.394, -0.000]
Epoch 1 [7/172] - Loss: 96.624 [-96.357, 0.267, -0.000]
Epoch 1 [8/172] - Loss: 80.956 [-80.752, 0.204, -0.000]
Epoch 1 [9/172] - Loss: 69.646 [-69.420, 0.226, -0.000]
Epoch 1 [10/172] - Loss: 68.924 [-68.710, 0.214, -0.000]
Epoch 1 [11/172] - Loss: 64.165 [-63.946, 0.219, -0.000]
Epoch 1 [12/172] - Loss: 58.513 [-58.327, 0.186, -0.000]
Epoch 1 [13/172] - Loss: 52.696 [-52.519, 0.176, -0.000]
Epoch 1 [14/172] - Loss: 49.839 [-49.701, 0.138, -0.000]
Epoch 1 [15/172] - Loss: 48.205 [-48.083, 0.122, -0.000]
Epoch 1 [16/172] - Loss: 42.446 [-42.348, 0.097, -0.000]
Epoch 1 [17/172] - Loss: 38.5

Epoch 1 [148/172] - Loss: 5.267 [-5.265, 0.003, -0.000]
Epoch 1 [149/172] - Loss: 5.167 [-5.165, 0.003, -0.000]
Epoch 1 [150/172] - Loss: 5.099 [-5.096, 0.003, -0.000]
Epoch 1 [151/172] - Loss: 5.061 [-5.059, 0.003, -0.000]
Epoch 1 [152/172] - Loss: 5.291 [-5.288, 0.003, -0.000]
Epoch 1 [153/172] - Loss: 5.101 [-5.098, 0.003, -0.000]
Epoch 1 [154/172] - Loss: 5.121 [-5.119, 0.003, -0.000]
Epoch 1 [155/172] - Loss: 4.892 [-4.889, 0.003, -0.000]
Epoch 1 [156/172] - Loss: 4.994 [-4.992, 0.003, -0.000]
Epoch 1 [157/172] - Loss: 5.229 [-5.227, 0.002, -0.000]
Epoch 1 [158/172] - Loss: 4.926 [-4.923, 0.003, -0.000]
Epoch 1 [159/172] - Loss: 5.028 [-5.026, 0.003, -0.000]
Epoch 1 [160/172] - Loss: 4.975 [-4.973, 0.002, -0.000]
Epoch 1 [161/172] - Loss: 5.083 [-5.081, 0.002, -0.000]
Epoch 1 [162/172] - Loss: 5.236 [-5.234, 0.002, -0.000]
Epoch 1 [163/172] - Loss: 5.302 [-5.299, 0.002, -0.000]
Epoch 1 [164/172] - Loss: 5.162 [-5.159, 0.002, -0.000]
Epoch 1 [165/172] - Loss: 5.098 [-5.096, 0.002, 

Epoch 2 [125/172] - Loss: 4.161 [-4.159, 0.002, -0.000]
Epoch 2 [126/172] - Loss: 4.300 [-4.299, 0.002, -0.000]
Epoch 2 [127/172] - Loss: 4.352 [-4.350, 0.002, -0.000]
Epoch 2 [128/172] - Loss: 4.249 [-4.248, 0.002, -0.000]
Epoch 2 [129/172] - Loss: 4.311 [-4.309, 0.002, -0.000]
Epoch 2 [130/172] - Loss: 4.268 [-4.266, 0.002, -0.000]
Epoch 2 [131/172] - Loss: 4.286 [-4.284, 0.002, -0.000]
Epoch 2 [132/172] - Loss: 4.346 [-4.344, 0.002, -0.000]
Epoch 2 [133/172] - Loss: 4.267 [-4.265, 0.002, -0.000]
Epoch 2 [134/172] - Loss: 4.237 [-4.235, 0.002, -0.000]
Epoch 2 [135/172] - Loss: 4.275 [-4.273, 0.002, -0.000]
Epoch 2 [136/172] - Loss: 4.207 [-4.205, 0.002, -0.000]
Epoch 2 [137/172] - Loss: 4.341 [-4.340, 0.002, -0.000]
Epoch 2 [138/172] - Loss: 4.300 [-4.299, 0.002, -0.000]
Epoch 2 [139/172] - Loss: 4.266 [-4.264, 0.002, -0.000]
Epoch 2 [140/172] - Loss: 4.249 [-4.247, 0.002, -0.000]
Epoch 2 [141/172] - Loss: 4.284 [-4.282, 0.002, -0.000]
Epoch 2 [142/172] - Loss: 4.171 [-4.169, 0.002, 

Epoch 3 [103/172] - Loss: 4.125 [-4.124, 0.002, -0.000]
Epoch 3 [104/172] - Loss: 4.100 [-4.098, 0.002, -0.000]
Epoch 3 [105/172] - Loss: 4.225 [-4.224, 0.002, -0.000]
Epoch 3 [106/172] - Loss: 4.151 [-4.150, 0.002, -0.000]
Epoch 3 [107/172] - Loss: 4.116 [-4.114, 0.002, -0.000]
Epoch 3 [108/172] - Loss: 4.116 [-4.114, 0.002, -0.000]
Epoch 3 [109/172] - Loss: 4.056 [-4.054, 0.002, -0.000]
Epoch 3 [110/172] - Loss: 4.107 [-4.106, 0.002, -0.000]
Epoch 3 [111/172] - Loss: 4.082 [-4.080, 0.002, -0.000]
Epoch 3 [112/172] - Loss: 4.147 [-4.145, 0.002, -0.000]
Epoch 3 [113/172] - Loss: 4.177 [-4.175, 0.002, -0.000]
Epoch 3 [114/172] - Loss: 4.081 [-4.079, 0.002, -0.000]
Epoch 3 [115/172] - Loss: 4.158 [-4.156, 0.002, -0.000]
Epoch 3 [116/172] - Loss: 4.088 [-4.086, 0.002, -0.000]
Epoch 3 [117/172] - Loss: 4.114 [-4.112, 0.001, -0.000]
Epoch 3 [118/172] - Loss: 4.124 [-4.122, 0.002, -0.000]
Epoch 3 [119/172] - Loss: 4.199 [-4.197, 0.002, -0.000]
Epoch 3 [120/172] - Loss: 4.247 [-4.246, 0.002, 

Epoch 4 [79/172] - Loss: 4.051 [-4.050, 0.001, -0.000]
Epoch 4 [80/172] - Loss: 4.049 [-4.048, 0.002, -0.000]
Epoch 4 [81/172] - Loss: 4.068 [-4.067, 0.001, -0.000]
Epoch 4 [82/172] - Loss: 4.070 [-4.068, 0.002, -0.000]
Epoch 4 [83/172] - Loss: 3.996 [-3.995, 0.001, -0.000]
Epoch 4 [84/172] - Loss: 4.062 [-4.060, 0.001, -0.000]
Epoch 4 [85/172] - Loss: 4.043 [-4.042, 0.001, -0.000]
Epoch 4 [86/172] - Loss: 4.027 [-4.025, 0.001, -0.000]
Epoch 4 [87/172] - Loss: 3.994 [-3.992, 0.001, -0.000]
Epoch 4 [88/172] - Loss: 4.060 [-4.059, 0.002, -0.000]
Epoch 4 [89/172] - Loss: 3.987 [-3.986, 0.001, -0.000]
Epoch 4 [90/172] - Loss: 3.994 [-3.992, 0.002, -0.000]
Epoch 4 [91/172] - Loss: 3.990 [-3.989, 0.001, -0.000]
Epoch 4 [92/172] - Loss: 3.996 [-3.995, 0.002, -0.000]
Epoch 4 [93/172] - Loss: 4.094 [-4.093, 0.001, -0.000]
Epoch 4 [94/172] - Loss: 3.989 [-3.987, 0.001, -0.000]
Epoch 4 [95/172] - Loss: 4.036 [-4.034, 0.001, -0.000]
Epoch 4 [96/172] - Loss: 3.999 [-3.998, 0.001, -0.000]
Epoch 4 [9

Epoch 5 [56/172] - Loss: 3.938 [-3.937, 0.001, -0.000]
Epoch 5 [57/172] - Loss: 4.047 [-4.045, 0.002, -0.000]
Epoch 5 [58/172] - Loss: 3.998 [-3.997, 0.001, -0.000]
Epoch 5 [59/172] - Loss: 4.012 [-4.011, 0.001, -0.000]
Epoch 5 [60/172] - Loss: 4.035 [-4.034, 0.001, -0.000]
Epoch 5 [61/172] - Loss: 4.043 [-4.041, 0.001, -0.000]
Epoch 5 [62/172] - Loss: 3.966 [-3.965, 0.001, -0.000]
Epoch 5 [63/172] - Loss: 4.078 [-4.076, 0.001, -0.000]
Epoch 5 [64/172] - Loss: 3.998 [-3.996, 0.001, -0.000]
Epoch 5 [65/172] - Loss: 3.991 [-3.989, 0.001, -0.000]
Epoch 5 [66/172] - Loss: 4.044 [-4.042, 0.001, -0.000]
Epoch 5 [67/172] - Loss: 4.013 [-4.012, 0.001, -0.000]
Epoch 5 [68/172] - Loss: 3.999 [-3.997, 0.001, -0.000]
Epoch 5 [69/172] - Loss: 4.002 [-4.000, 0.002, -0.000]
Epoch 5 [70/172] - Loss: 4.027 [-4.026, 0.001, -0.000]
Epoch 5 [71/172] - Loss: 3.997 [-3.996, 0.001, -0.000]
Epoch 5 [72/172] - Loss: 4.007 [-4.006, 0.001, -0.000]
Epoch 5 [73/172] - Loss: 4.051 [-4.050, 0.001, -0.000]
Epoch 5 [7

Epoch 6 [33/172] - Loss: 4.025 [-4.023, 0.001, -0.000]
Epoch 6 [34/172] - Loss: 3.961 [-3.960, 0.001, -0.000]
Epoch 6 [35/172] - Loss: 4.097 [-4.096, 0.001, -0.000]
Epoch 6 [36/172] - Loss: 4.028 [-4.026, 0.001, -0.000]
Epoch 6 [37/172] - Loss: 4.013 [-4.012, 0.001, -0.000]
Epoch 6 [38/172] - Loss: 3.978 [-3.977, 0.001, -0.000]
Epoch 6 [39/172] - Loss: 4.039 [-4.038, 0.001, -0.000]
Epoch 6 [40/172] - Loss: 3.983 [-3.981, 0.001, -0.000]
Epoch 6 [41/172] - Loss: 4.011 [-4.010, 0.001, -0.000]
Epoch 6 [42/172] - Loss: 3.999 [-3.998, 0.002, -0.000]
Epoch 6 [43/172] - Loss: 4.014 [-4.012, 0.001, -0.000]
Epoch 6 [44/172] - Loss: 3.998 [-3.997, 0.001, -0.000]
Epoch 6 [45/172] - Loss: 3.966 [-3.964, 0.001, -0.000]
Epoch 6 [46/172] - Loss: 4.007 [-4.006, 0.001, -0.000]
Epoch 6 [47/172] - Loss: 4.000 [-3.999, 0.001, -0.000]
Epoch 6 [48/172] - Loss: 4.072 [-4.071, 0.001, -0.000]
Epoch 6 [49/172] - Loss: 4.010 [-4.008, 0.001, -0.000]
Epoch 6 [50/172] - Loss: 4.008 [-4.007, 0.001, -0.000]
Epoch 6 [5

Epoch 7 [9/172] - Loss: 3.994 [-3.992, 0.001, -0.000]
Epoch 7 [10/172] - Loss: 3.999 [-3.997, 0.001, -0.000]
Epoch 7 [11/172] - Loss: 3.977 [-3.976, 0.001, -0.000]
Epoch 7 [12/172] - Loss: 3.984 [-3.982, 0.001, -0.000]
Epoch 7 [13/172] - Loss: 4.013 [-4.011, 0.001, -0.000]
Epoch 7 [14/172] - Loss: 3.937 [-3.936, 0.001, -0.000]
Epoch 7 [15/172] - Loss: 3.962 [-3.961, 0.002, -0.000]
Epoch 7 [16/172] - Loss: 3.989 [-3.988, 0.001, -0.000]
Epoch 7 [17/172] - Loss: 4.020 [-4.018, 0.001, -0.000]
Epoch 7 [18/172] - Loss: 3.963 [-3.961, 0.002, -0.000]
Epoch 7 [19/172] - Loss: 4.032 [-4.031, 0.001, -0.000]
Epoch 7 [20/172] - Loss: 4.005 [-4.004, 0.001, -0.000]
Epoch 7 [21/172] - Loss: 3.990 [-3.989, 0.002, -0.000]
Epoch 7 [22/172] - Loss: 4.019 [-4.018, 0.001, -0.000]
Epoch 7 [23/172] - Loss: 4.034 [-4.032, 0.002, -0.000]
Epoch 7 [24/172] - Loss: 4.011 [-4.009, 0.001, -0.000]
Epoch 7 [25/172] - Loss: 3.997 [-3.996, 0.001, -0.000]
Epoch 7 [26/172] - Loss: 4.000 [-3.999, 0.001, -0.000]
Epoch 7 [27

Epoch 7 [157/172] - Loss: 4.021 [-4.019, 0.001, -0.000]
Epoch 7 [158/172] - Loss: 3.959 [-3.957, 0.001, -0.000]
Epoch 7 [159/172] - Loss: 4.020 [-4.018, 0.001, -0.000]
Epoch 7 [160/172] - Loss: 4.016 [-4.014, 0.001, -0.000]
Epoch 7 [161/172] - Loss: 3.991 [-3.989, 0.001, -0.000]
Epoch 7 [162/172] - Loss: 3.974 [-3.973, 0.001, -0.000]
Epoch 7 [163/172] - Loss: 3.961 [-3.960, 0.001, -0.000]
Epoch 7 [164/172] - Loss: 3.990 [-3.988, 0.001, -0.000]
Epoch 7 [165/172] - Loss: 3.996 [-3.995, 0.001, -0.000]
Epoch 7 [166/172] - Loss: 3.992 [-3.990, 0.001, -0.000]
Epoch 7 [167/172] - Loss: 3.982 [-3.981, 0.001, -0.000]
Epoch 7 [168/172] - Loss: 3.989 [-3.988, 0.002, -0.000]
Epoch 7 [169/172] - Loss: 3.918 [-3.917, 0.001, -0.000]
Epoch 7 [170/172] - Loss: 3.970 [-3.969, 0.002, -0.000]
Epoch 7 [171/172] - Loss: 4.001 [-3.999, 0.001, -0.000]


## Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`) using the standard SKI testing code, with no acceleration or precomputation. Because the test set is substantially smaller than the training set, we don't need to make predictions in mini batches here, although our other tutorials demonstrate how to do this (for example, see the CIFAR tutorial).

In [8]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False):
    preds = model(test_x)

In [9]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 9.33540153503418
